In [1]:
"""
A 1000 run of the gillespied function, with constant inputs 
"""

"A 1000 run of the gillespied function, with constant inputs \n"

In [2]:
using Distributed, Plots, DelimitedFiles, KernelDensity

In [3]:
length(Sys.cpu_info())
addprocs(4) ;

In [4]:
@everywhere using Random, Distributions

In [5]:
@everywhere struct SPN
    init::Vector{Real}
    rates::Vector{Real}
    Stoichiometry_matrix::Array{Real}
    function SPN(init, rates, Stoichiometry_matrix)
        new(init, rates, Stoichiometry_matrix)
    end
end

In [6]:
@everywhere init(N::SPN) = Float64.(N.init)
@everywhere rates(N::SPN) = Float64.(N.rates)
@everywhere StoiMat(N::SPN) = Float64.(N.Stoichiometry_matrix)

In [7]:
@everywhere function hazard(x::Vector{Float64}, th::Vector{Float64}, c0::Float64)::Vector{Float64}
    error = sum(x) - c0
    W = x[1]
    M = x[2]
    k = th[1:5]
    Kc = th[6:7]
    if error < 0.0
        k1 = k[1]-error*Kc[1]
        k2 = k[2]-error*Kc[1]
        return [W,M,W,M,W].*[k1, k2, k[3], k[4], k[5]]
    else 
        k1 = 2*k[1]/(1+exp(-error*Kc[2]))
        k2 = 2*k[2]/(1+exp(-error*Kc[2]))
        return [W,M,W,M,W].*[k1, k2, k[3], k[4], k[5]]
    end
end

In [8]:
@everywhere function hazzy(x::Vector{Float64}, th::Vector{Float64})::Vector{Float64}
    return [x[1], x[2], x[1], x[2], x[1]].*th[1:5]
end

In [9]:
@everywhere const post = [[2,0,0,0,1] [0,2,0,0,1]]
@everywhere const pre = [[1,0,1,0,1] [0,1,0,1,0]]
@everywhere const S = post - pre
@everywhere const k = [3.06e-8, 3.06e-8, 3.06e-8, 3.06e-8, 0.0, 8.99e-9, 2e-3];

In [10]:
@everywhere function gen_inits(μ::Real, σ::Real, α::Real, β::Real)::Vector{Float64}
    CC = rand(Normal(μ, σ))
    hh = rand(Beta(α, β))
    return round.( [CC*(1-hh), CC*hh] )
end

In [11]:
@everywhere function trans_summ(popdym)
    copy_num = popdym[:,1] .+ popdym[:,2]
    
    mut_load = Array{Union{Float64, Missing}}(undef, length(copy_num))
    idx = copy_num .!= 0.0
    mut_load[.!idx] .= missing
    mut_load[idx] .=  popdym[idx,2] ./ copy_num[idx]
    
    return hcat(copy_num, mut_load)
end

In [12]:
@everywhere function gillespied(N::SPN, Tmax::Real, dtout::Real)::Array{Union{Float64, Missing}}
    k = rates(N)
    x = init(N)
    Nout = trunc(Int, Tmax/dtout)
    S = StoiMat(N)
    xmat = Array{Union{Float64, Missing}}(undef, (2,Nout+1))
    xmat[:,1] = x
    i = 1
    tt = 0.0
    target = 0.0
    c0 = sum(x)
    while i <= Nout
        h = hazard(x, k, c0)
        h0 = sum(h)
        if h0<1e-10
            xmat[:,i:(Nout+1)] = zeros(2,Nout-i+2)
            return trans_summ(xmat')
        else
            Exp = Exponential(1/h0)
            tt = tt + rand(Exp)
        end
        while tt>=target && i<=Nout
            i += 1
            xmat[:,i] = x
            target += dtout
        end
        Cat = Categorical(h/h0)
        r = rand(Cat)
        x += S'[:,r]
        println(x)
    end
    return trans_summ(xmat')
end

In [13]:
@everywhere function quantiles(sims, p)
    """
    returns quantile summaries from simulations
    """
    Nsim = size(sims)[3] # Nsim: number of simulations
    n = size(sims)[1] # length of one simulation
    out = Array{Float64}(undef, n,length(p),2)
    for t=1:n
        out[t,:,1] = quantile(skipmissing([sims[t,1,i] for i=1:Nsim]), p)
        out[t,:,2] = quantile(skipmissing([sims[t,2,i] for i=1:Nsim]), p)
    end
    out
end

In [14]:
Nsim = 1000
Tmax = 80*365*24*3600
step_out = 7*24*3600 ; 

In [15]:
Ntest = SPN([100,100], k, S)
@time gillespied(Ntest, Tmax, step_out);
"""
a single run takes ~0.03 seconds
"""

[99.0, 100.0]
[100.0, 100.0]
[100.0, 101.0]
[100.0, 100.0]
[100.0, 101.0]
[100.0, 100.0]
[100.0, 101.0]
[100.0, 102.0]
[100.0, 103.0]
[100.0, 102.0]
[100.0, 101.0]
[101.0, 101.0]
[102.0, 101.0]
[103.0, 101.0]
[103.0, 102.0]
[102.0, 102.0]
[103.0, 102.0]
[104.0, 102.0]
[104.0, 101.0]
[103.0, 101.0]
[102.0, 101.0]
[102.0, 100.0]
[103.0, 100.0]
[103.0, 99.0]
[103.0, 100.0]
[102.0, 100.0]
[103.0, 100.0]
[104.0, 100.0]
[104.0, 99.0]
[104.0, 98.0]
[104.0, 99.0]
[105.0, 99.0]
[105.0, 98.0]
[104.0, 98.0]
[105.0, 98.0]
[106.0, 98.0]
[105.0, 98.0]
[105.0, 97.0]
[105.0, 98.0]
[106.0, 98.0]
[105.0, 98.0]
[104.0, 98.0]
[104.0, 99.0]
[104.0, 100.0]
[105.0, 100.0]
[104.0, 100.0]
[103.0, 100.0]
[103.0, 101.0]
[103.0, 100.0]
[104.0, 100.0]
[104.0, 99.0]
[104.0, 98.0]
[105.0, 98.0]
[104.0, 98.0]
[105.0, 98.0]
[104.0, 98.0]
[104.0, 97.0]
[105.0, 97.0]
[105.0, 96.0]
[104.0, 96.0]
[105.0, 96.0]
[105.0, 97.0]
[105.0, 96.0]
[105.0, 97.0]
[105.0, 96.0]
[105.0, 95.0]
[104.0, 95.0]
[104.0, 94.0]
[103.0, 94.0]
[

[98.0, 109.0]
[97.0, 109.0]
[97.0, 110.0]
[97.0, 109.0]
[97.0, 108.0]
[97.0, 107.0]
[97.0, 106.0]
[98.0, 106.0]
[97.0, 106.0]
[97.0, 107.0]
[97.0, 106.0]
[96.0, 106.0]
[97.0, 106.0]
[97.0, 107.0]
[96.0, 107.0]
[96.0, 108.0]
[97.0, 108.0]
[96.0, 108.0]
[96.0, 107.0]
[96.0, 108.0]
[96.0, 107.0]
[96.0, 108.0]
[96.0, 107.0]
[95.0, 107.0]
[94.0, 107.0]
[94.0, 106.0]
[93.0, 106.0]
[93.0, 107.0]
[92.0, 107.0]
[92.0, 106.0]
[92.0, 107.0]
[92.0, 108.0]
[91.0, 108.0]
[92.0, 108.0]
[92.0, 109.0]
[93.0, 109.0]
[92.0, 109.0]
[91.0, 109.0]
[90.0, 109.0]
[91.0, 109.0]
[92.0, 109.0]
[91.0, 109.0]
[90.0, 109.0]
[89.0, 109.0]
[88.0, 109.0]
[89.0, 109.0]
[90.0, 109.0]
[90.0, 110.0]
[90.0, 111.0]
[89.0, 111.0]
[89.0, 112.0]
[89.0, 113.0]
[89.0, 114.0]
[89.0, 113.0]
[89.0, 114.0]
[90.0, 114.0]
[91.0, 114.0]
[92.0, 114.0]
[93.0, 114.0]
[94.0, 114.0]
[93.0, 114.0]
[94.0, 114.0]
[94.0, 113.0]
[93.0, 113.0]
[93.0, 112.0]
[92.0, 112.0]
[91.0, 112.0]
[90.0, 112.0]
[89.0, 112.0]
[89.0, 111.0]
[89.0, 112.0]
[90.0,

[64.0, 142.0]
[64.0, 143.0]
[64.0, 142.0]
[63.0, 142.0]
[63.0, 141.0]
[63.0, 140.0]
[63.0, 139.0]
[63.0, 140.0]
[63.0, 139.0]
[63.0, 138.0]
[63.0, 139.0]
[63.0, 138.0]
[64.0, 138.0]
[64.0, 139.0]
[64.0, 140.0]
[64.0, 139.0]
[65.0, 139.0]
[65.0, 138.0]
[65.0, 137.0]
[66.0, 137.0]
[67.0, 137.0]
[68.0, 137.0]
[68.0, 138.0]
[68.0, 137.0]
[67.0, 137.0]
[68.0, 137.0]
[67.0, 137.0]
[67.0, 138.0]
[66.0, 138.0]
[66.0, 137.0]
[66.0, 138.0]
[66.0, 137.0]
[66.0, 138.0]
[67.0, 138.0]
[68.0, 138.0]
[68.0, 137.0]
[67.0, 137.0]
[67.0, 136.0]
[67.0, 135.0]
[67.0, 136.0]
[67.0, 135.0]
[67.0, 134.0]
[66.0, 134.0]
[66.0, 133.0]
[65.0, 133.0]
[65.0, 134.0]
[66.0, 134.0]
[66.0, 133.0]
[66.0, 134.0]
[66.0, 133.0]
[66.0, 134.0]
[66.0, 133.0]
[65.0, 133.0]
[65.0, 134.0]
[64.0, 134.0]
[65.0, 134.0]
[65.0, 135.0]
[65.0, 134.0]
[65.0, 133.0]
[65.0, 134.0]
[66.0, 134.0]
[67.0, 134.0]
[68.0, 134.0]
[69.0, 134.0]
[69.0, 135.0]
[70.0, 135.0]
[69.0, 135.0]
[69.0, 134.0]
[69.0, 133.0]
[68.0, 133.0]
[68.0, 134.0]
[68.0,

[83.0, 125.0]
[84.0, 125.0]
[84.0, 126.0]
[84.0, 125.0]
[84.0, 126.0]
[84.0, 127.0]
[84.0, 128.0]
[84.0, 129.0]
[84.0, 130.0]
[84.0, 131.0]
[83.0, 131.0]
[83.0, 132.0]
[83.0, 133.0]
[83.0, 134.0]
[84.0, 134.0]
[83.0, 134.0]
[82.0, 134.0]
[82.0, 135.0]
[82.0, 136.0]
[82.0, 135.0]
[82.0, 136.0]
[81.0, 136.0]
[80.0, 136.0]
[80.0, 135.0]
[80.0, 134.0]
[80.0, 133.0]
[80.0, 134.0]
[80.0, 133.0]
[81.0, 133.0]
[81.0, 134.0]
[81.0, 133.0]
[82.0, 133.0]
[82.0, 134.0]
[82.0, 133.0]
[82.0, 132.0]
[82.0, 131.0]
[82.0, 130.0]
[83.0, 130.0]
[84.0, 130.0]
[84.0, 129.0]
[85.0, 129.0]
[85.0, 130.0]
[85.0, 129.0]
[85.0, 128.0]
[85.0, 129.0]
[85.0, 128.0]
[85.0, 129.0]
[85.0, 130.0]
[85.0, 131.0]
[85.0, 130.0]
[85.0, 129.0]
[85.0, 128.0]
[86.0, 128.0]
[86.0, 129.0]
[86.0, 128.0]
[86.0, 127.0]
[86.0, 128.0]
[87.0, 128.0]
[87.0, 127.0]
[87.0, 128.0]
[87.0, 129.0]
[87.0, 128.0]
[88.0, 128.0]
[88.0, 129.0]
[87.0, 129.0]
[87.0, 130.0]
[88.0, 130.0]
[88.0, 131.0]
[88.0, 132.0]
[89.0, 132.0]
[90.0, 132.0]
[91.0,

[140.0, 194.0]
[140.0, 195.0]
[140.0, 194.0]
[139.0, 194.0]
[139.0, 193.0]
[140.0, 193.0]
[139.0, 193.0]
[139.0, 192.0]
[139.0, 193.0]
[140.0, 193.0]
[139.0, 193.0]
[139.0, 194.0]
[140.0, 194.0]
[140.0, 193.0]
[140.0, 192.0]
[140.0, 193.0]
[140.0, 194.0]
[140.0, 193.0]
[141.0, 193.0]
[140.0, 193.0]
[140.0, 194.0]
[140.0, 195.0]
[140.0, 196.0]
[140.0, 197.0]
[140.0, 196.0]
[140.0, 195.0]
[139.0, 195.0]
[138.0, 195.0]
[138.0, 196.0]
[139.0, 196.0]
[138.0, 196.0]
[138.0, 197.0]
[139.0, 197.0]
[138.0, 197.0]
[138.0, 198.0]
[138.0, 197.0]
[139.0, 197.0]
[139.0, 198.0]
[139.0, 197.0]
[140.0, 197.0]
[140.0, 198.0]
[140.0, 197.0]
[140.0, 198.0]
[140.0, 197.0]
[141.0, 197.0]
[141.0, 196.0]
[141.0, 195.0]
[142.0, 195.0]
[142.0, 194.0]
[142.0, 193.0]
[142.0, 194.0]
[142.0, 193.0]
[143.0, 193.0]
[143.0, 194.0]
[143.0, 195.0]
[143.0, 196.0]
[143.0, 195.0]
[143.0, 196.0]
[144.0, 196.0]
[144.0, 195.0]
[144.0, 194.0]
[145.0, 194.0]
[145.0, 193.0]
[144.0, 193.0]
[144.0, 192.0]
[143.0, 192.0]
[142.0, 19

...

[4259.0, 4440.0]
[4259.0, 4441.0]
[4259.0, 4440.0]
[4259.0, 4441.0]
[4260.0, 4441.0]
[4260.0, 4440.0]
[4260.0, 4439.0]
[4260.0, 4440.0]
[4261.0, 4440.0]
[4261.0, 4441.0]
[4261.0, 4442.0]
[4261.0, 4443.0]
[4260.0, 4443.0]
[4260.0, 4444.0]
[4260.0, 4445.0]
[4261.0, 4445.0]
[4261.0, 4444.0]
[4262.0, 4444.0]
[4262.0, 4443.0]
[4262.0, 4444.0]
[4262.0, 4445.0]
[4262.0, 4444.0]
[4262.0, 4445.0]
[4262.0, 4446.0]
[4263.0, 4446.0]
[4263.0, 4447.0]
[4263.0, 4448.0]
[4264.0, 4448.0]
[4265.0, 4448.0]
[4265.0, 4447.0]
[4265.0, 4446.0]
[4265.0, 4445.0]
[4265.0, 4446.0]
[4266.0, 4446.0]
[4266.0, 4447.0]
[4266.0, 4446.0]
[4265.0, 4446.0]
[4264.0, 4446.0]
[4264.0, 4445.0]
[4264.0, 4444.0]
[4265.0, 4444.0]
[4265.0, 4443.0]
[4265.0, 4444.0]
[4266.0, 4444.0]
[4267.0, 4444.0]
[4268.0, 4444.0]
[4268.0, 4445.0]
[4268.0, 4446.0]
[4269.0, 4446.0]
[4269.0, 4447.0]
[4270.0, 4447.0]
[4270.0, 4448.0]
[4271.0, 4448.0]
[4271.0, 4447.0]
[4271.0, 4448.0]
[4272.0, 4448.0]
[4272.0, 4449.0]
[4272.0, 4450.0]
[4273.0, 4450.

[4390.0, 4526.0]
[4389.0, 4526.0]
[4389.0, 4527.0]
[4389.0, 4526.0]
[4389.0, 4527.0]
[4389.0, 4528.0]
[4389.0, 4529.0]
[4390.0, 4529.0]
[4391.0, 4529.0]
[4392.0, 4529.0]
[4391.0, 4529.0]
[4392.0, 4529.0]
[4392.0, 4530.0]
[4392.0, 4531.0]
[4393.0, 4531.0]
[4393.0, 4532.0]
[4393.0, 4533.0]
[4393.0, 4532.0]
[4392.0, 4532.0]
[4392.0, 4531.0]
[4392.0, 4532.0]
[4392.0, 4533.0]
[4391.0, 4533.0]
[4391.0, 4532.0]
[4391.0, 4533.0]
[4391.0, 4534.0]
[4390.0, 4534.0]
[4390.0, 4533.0]
[4389.0, 4533.0]
[4390.0, 4533.0]
[4389.0, 4533.0]
[4390.0, 4533.0]
[4391.0, 4533.0]
[4391.0, 4534.0]
[4392.0, 4534.0]
[4392.0, 4535.0]
[4393.0, 4535.0]
[4394.0, 4535.0]
[4395.0, 4535.0]
[4395.0, 4534.0]
[4395.0, 4535.0]
[4395.0, 4534.0]
[4396.0, 4534.0]
[4397.0, 4534.0]
[4397.0, 4533.0]
[4397.0, 4532.0]
[4397.0, 4531.0]
[4397.0, 4532.0]
[4398.0, 4532.0]
[4398.0, 4531.0]
[4399.0, 4531.0]
[4399.0, 4532.0]
[4400.0, 4532.0]
[4400.0, 4533.0]
[4401.0, 4533.0]
[4400.0, 4533.0]
[4401.0, 4533.0]
[4401.0, 4532.0]
[4401.0, 4531.

Excessive output truncated after 524291 bytes.

[4404.0, 4536.0]
[4404.0, 4537.0]
[4404.0, 4538.0]
[4404.0, 4539.0]
[4405.0, 4539.0]
[4406.0, 4539.0]
[4406.0, 4540.0]
[4406.0, 4539.0]
[4406.0, 4540.0]
[4407.0, 4540.0]
[4406.0, 4540.0]
[4407.0, 4540.0]
[4406.0, 4540.0]
[4405.0, 4540.0]
[4405.0, 4541.0]
[4405.0, 4542.0]
[4405.0, 4543.0]
[4405.0, 4544.0]
[4405.0, 4543.0]
[4406.0, 4543.0]
[4407.0, 4543.0]
[4407.0, 4542.0]
[4408.0, 4542.0]
[4408.0, 4541.0]
[4408.0, 4542.0]
[4408.0, 4543.0]
[4409.0, 4543.0]
[4410.0, 4543.0]
[4410.0, 4544.0]
[4411.0, 4544.0]
[4411.0, 4545.0]
[4412.0, 4545.0]
[4412.0, 4546.0]
[4412.0, 4545.0]
[4412.0, 4546.0]
[4412.0, 4547.0]
[4411.0, 4547.0]
[4412.0, 4547.0]
[4413.0, 4547.0]
[4414.0, 4547.0]
[4414.0, 4548.0]
[4413.0, 4548.0]
[4412.0, 4548.0]
[4413.0, 4548.0]
[4412.0, 4548.0]
[4412.0, 4549.0]
[4411.0, 4549.0]
[4412.0, 4549.0]
[4411.0, 4549.0]
[4411.0, 4550.0]
[4410.0, 4550.0]
[4410.0, 4549.0]
[4411.0, 4549.0]
[4412.0, 4549.0]
[4413.0, 4549.0]
[4413.0, 4550.0]
[4413.0, 4551.0]
[4414.0, 4551.0]
[4414.0, 4552.

LoadError: InterruptException:

In [16]:
"""
simulations_single = Array{Float64}(undef, n(Ntest), 2, Nsim)
@time for i=1:Nsim
    simulations_single[:,:,i] = gillespied(Ntest)
end
This is a lot slower (doubly slow) than the previous version without the SPN structure
For 1000 simulations 26.536 seconds
"""

"simulations_single = Array{Float64}(undef, n(Ntest), 2, Nsim)\n@time for i=1:Nsim\n    simulations_single[:,:,i] = gillespied(Ntest)\nend\nThis is a lot slower (doubly slow) than the previous version without the SPN structure\nFor 1000 simulations 26.536 seconds\n"

In [17]:
#summ_single = raw_to_summ(simulations_single);
#qnts_single = quantiles(summ_single, [0.025,0.1,0.5,0.9,0.975]) ;

In [18]:
# The arguments are: 1) a function 'f' and 2) a list with the input.
@everywhere function par_map(Nsim, f, SPN, Tmax, dtout)
    np = nworkers()            # Number of processes available.
    Nout = trunc(Int, Tmax/dtout) + 1 # dimension for output
    output = Array{Union{Float64, Missing}}(undef, Nout,2,Nsim) # Where we will write the results. As we do not know
                             # the type (Integer, Tuple...) we write "Any"
    i = 1
    nextidx() = (idx = i; i += 1; idx) # Function to know which is the next work item.
                                       # In this case it is just an index.
    @sync begin #@sync: must complete all jobs in block
        for p = 1:np # loops through all processes (workers)
            if p != myid() || np == 1 # first worker used only if all others are busy 
                @async begin # launch several tasks simultaneaously
                    while true
                        idx = nextidx()
                        if idx > Nsim
                            break
                        end
                        output[:,:,idx] = remotecall_fetch(f, p, SPN, Tmax, dtout)
                    end
                end
            end
        end
    end
    output
end

In [ ]:
@time simulations = par_map(Nsim, gillespied, Ntest, Tmax, step_out) ;
"""
simple simulation: ~ 7.5 seconds
4 workers, 1000 simulations
"""

      From worker 4:	[100.0, 99.0]
      From worker 3:	[101.0, 100.0]
      From worker 3:	[102.0, 100.0]
      From worker 3:	[103.0, 100.0]
      From worker 3:	[103.0, 101.0]
      From worker 3:	[103.0, 102.0]
      From worker 3:	[102.0, 102.0]
      From worker 3:	[103.0, 102.0]
      From worker 3:	[103.0, 101.0]
      From worker 3:	[103.0, 100.0]
      From worker 3:	[103.0, 99.0]
      From worker 3:	[102.0, 99.0]
      From worker 3:	[102.0, 98.0]
      From worker 3:	[101.0, 98.0]
      From worker 3:	[101.0, 97.0]
      From worker 3:	[100.0, 97.0]
      From worker 3:	[100.0, 98.0]
      From worker 3:	[99.0, 98.0]
      From worker 3:	[100.0, 98.0]
      From worker 3:	[99.0, 98.0]
      From worker 3:	[100.0, 98.0]
      From worker 3:	[101.0, 98.0]
      From worker 3:	[102.0, 98.0]
      From worker 3:	[102.0, 99.0]
      From worker 3:	[103.0, 99.0]
      From worker 4:	[101.0, 99.0]
      From worker 4:	[101.0, 100.0]
      From worker 4:	[101.0, 101.0]
      From 

      From worker 3:	[107.0, 98.0]
      From worker 3:	[107.0, 99.0]
      From worker 3:	[107.0, 100.0]
      From worker 3:	[107.0, 101.0]
      From worker 2:	[98.0, 108.0]
      From worker 2:	[98.0, 107.0]
      From worker 2:	[98.0, 108.0]
      From worker 2:	[98.0, 107.0]
      From worker 4:	[111.0, 102.0]
      From worker 4:	[112.0, 102.0]
      From worker 4:	[112.0, 103.0]
      From worker 3:	[107.0, 100.0]
      From worker 2:	[98.0, 108.0]
      From worker 2:	[98.0, 109.0]
      From worker 4:	[112.0, 104.0]
      From worker 4:	[113.0, 104.0]
      From worker 4:	[112.0, 104.0]
      From worker 3:	[108.0, 100.0]
      From worker 2:	[98.0, 110.0]
      From worker 4:	[112.0, 103.0]
      From worker 3:	[108.0, 101.0]
      From worker 2:	[98.0, 109.0]
      From worker 3:	[108.0, 102.0]
      From worker 4:	[113.0, 103.0]
      From worker 2:	[98.0, 110.0]
      From worker 3:	[109.0, 102.0]
      From worker 2:	[98.0, 109.0]
      From worker 4:	[113.0, 102.0]
    

      From worker 3:	[109.0, 103.0]
      From worker 3:	[110.0, 103.0]
      From worker 3:	[110.0, 104.0]
      From worker 4:	[113.0, 97.0]
      From worker 4:	[114.0, 97.0]
      From worker 4:	[114.0, 96.0]
      From worker 2:	[107.0, 101.0]
      From worker 2:	[108.0, 101.0]
      From worker 3:	[109.0, 104.0]
      From worker 4:	[113.0, 96.0]
      From worker 4:	[112.0, 96.0]
      From worker 2:	[108.0, 100.0]
      From worker 2:	[109.0, 100.0]
      From worker 2:	[110.0, 100.0]
      From worker 3:	[110.0, 104.0]
      From worker 4:	[112.0, 95.0]
      From worker 2:	[110.0, 101.0]
      From worker 3:	[111.0, 104.0]
      From worker 3:	[110.0, 104.0]
      From worker 4:	[113.0, 95.0]
      From worker 2:	[109.0, 101.0]
      From worker 3:	[109.0, 104.0]
      From worker 4:	[113.0, 96.0]
      From worker 2:	[110.0, 101.0]
      From worker 3:	[109.0, 103.0]
      From worker 4:	[112.0, 96.0]
      From worker 2:	[110.0, 100.0]
      From worker 3:	[110.0, 103.0]
 

      From worker 4:	[112.0, 96.0]
      From worker 4:	[112.0, 97.0]
      From worker 4:	[111.0, 97.0]
      From worker 3:	[112.0, 119.0]
      From worker 3:	[113.0, 119.0]
      From worker 3:	[113.0, 118.0]
      From worker 2:	[118.0, 98.0]
      From worker 2:	[118.0, 99.0]
      From worker 2:	[117.0, 99.0]
      From worker 4:	[110.0, 97.0]
      From worker 3:	[114.0, 118.0]
      From worker 2:	[118.0, 99.0]
      From worker 2:	[119.0, 99.0]
      From worker 4:	[109.0, 97.0]
      From worker 4:	[108.0, 97.0]
      From worker 3:	[115.0, 118.0]
      From worker 2:	[119.0, 100.0]
      From worker 4:	[108.0, 98.0]
      From worker 3:	[115.0, 119.0]
      From worker 2:	[119.0, 101.0]
      From worker 3:	[115.0, 118.0]
      From worker 4:	[108.0, 99.0]
      From worker 2:	[119.0, 102.0]
      From worker 3:	[116.0, 118.0]
      From worker 4:	[109.0, 99.0]
      From worker 2:	[119.0, 101.0]
      From worker 4:	[108.0, 99.0]
      From worker 3:	[116.0, 117.0]
      F

      From worker 3:	[115.0, 110.0]
      From worker 3:	[115.0, 109.0]
      From worker 3:	[116.0, 109.0]
      From worker 3:	[116.0, 108.0]
      From worker 4:	[101.0, 101.0]
      From worker 4:	[101.0, 100.0]
      From worker 4:	[101.0, 99.0]
      From worker 2:	[120.0, 100.0]
      From worker 2:	[119.0, 100.0]
      From worker 2:	[120.0, 100.0]
      From worker 3:	[116.0, 107.0]
      From worker 4:	[101.0, 98.0]
      From worker 4:	[101.0, 99.0]
      From worker 2:	[120.0, 99.0]
      From worker 2:	[121.0, 99.0]
      From worker 3:	[117.0, 107.0]
      From worker 4:	[102.0, 99.0]
      From worker 2:	[120.0, 99.0]
      From worker 4:	[102.0, 98.0]
      From worker 3:	[117.0, 106.0]
      From worker 2:	[120.0, 98.0]
      From worker 3:	[117.0, 107.0]
      From worker 4:	[102.0, 99.0]
      From worker 2:	[120.0, 99.0]
      From worker 3:	[117.0, 106.0]
      From worker 4:	[102.0, 98.0]
      From worker 2:	[120.0, 100.0]
      From worker 3:	[117.0, 107.0]
    

...

      From worker 4:	[127.0, 220.0]
      From worker 4:	[126.0, 220.0]
      From worker 4:	[126.0, 221.0]
      From worker 2:	[122.0, 132.0]
      From worker 2:	[122.0, 133.0]
      From worker 4:	[126.0, 220.0]
      From worker 3:	[126.0, 93.0]
      From worker 3:	[126.0, 92.0]
      From worker 3:	[126.0, 91.0]
      From worker 2:	[122.0, 134.0]
      From worker 4:	[126.0, 221.0]
      From worker 3:	[126.0, 90.0]
      From worker 3:	[125.0, 90.0]
      From worker 2:	[121.0, 134.0]
      From worker 4:	[126.0, 222.0]
      From worker 3:	[125.0, 89.0]
      From worker 2:	[122.0, 134.0]
      From worker 2:	[121.0, 134.0]
      From worker 4:	[126.0, 221.0]
      From worker 2:	[121.0, 133.0]
      From worker 3:	[125.0, 88.0]
      From worker 3:	[126.0, 88.0]
      From worker 4:	[126.0, 222.0]
      From worker 2:	[122.0, 133.0]
      From worker 3:	[127.0, 88.0]
      From worker 4:	[126.0, 221.0]
      From worker 2:	[122.0, 132.0]
      From worker 3:	[128.0, 88.0]
  

      From worker 4:	[129.0, 231.0]
      From worker 3:	[120.0, 90.0]
      From worker 3:	[119.0, 90.0]
      From worker 3:	[119.0, 89.0]
      From worker 2:	[120.0, 128.0]
      From worker 2:	[120.0, 129.0]
      From worker 2:	[119.0, 129.0]
      From worker 4:	[130.0, 231.0]
      From worker 4:	[131.0, 231.0]
      From worker 4:	[130.0, 231.0]
      From worker 3:	[119.0, 90.0]
      From worker 2:	[120.0, 129.0]
      From worker 2:	[121.0, 129.0]
      From worker 4:	[131.0, 231.0]
      From worker 4:	[131.0, 232.0]
      From worker 3:	[119.0, 89.0]
      From worker 3:	[119.0, 90.0]
      From worker 2:	[122.0, 129.0]
      From worker 4:	[131.0, 231.0]
      From worker 3:	[119.0, 89.0]
      From worker 2:	[121.0, 129.0]
      From worker 4:	[132.0, 231.0]
      From worker 3:	[119.0, 88.0]
      From worker 2:	[120.0, 129.0]
      From worker 4:	[131.0, 231.0]
      From worker 2:	[120.0, 128.0]
      From worker 3:	[119.0, 87.0]
      From worker 4:	[132.0, 231.0]
 

      From worker 3:	[129.0, 92.0]
      From worker 3:	[129.0, 93.0]
      From worker 3:	[129.0, 94.0]
      From worker 3:	[128.0, 94.0]
      From worker 2:	[119.0, 121.0]
      From worker 2:	[119.0, 120.0]
      From worker 2:	[119.0, 119.0]
      From worker 4:	[136.0, 245.0]
      From worker 4:	[136.0, 246.0]
      From worker 4:	[135.0, 246.0]
      From worker 3:	[128.0, 95.0]
      From worker 2:	[119.0, 118.0]
      From worker 4:	[135.0, 245.0]
      From worker 4:	[135.0, 246.0]
      From worker 3:	[127.0, 95.0]
      From worker 2:	[118.0, 118.0]
      From worker 4:	[135.0, 245.0]
      From worker 2:	[119.0, 118.0]
      From worker 3:	[128.0, 95.0]
      From worker 2:	[119.0, 117.0]
      From worker 4:	[136.0, 245.0]
      From worker 3:	[127.0, 95.0]
      From worker 4:	[137.0, 245.0]
      From worker 2:	[120.0, 117.0]
      From worker 3:	[127.0, 96.0]
      From worker 4:	[137.0, 246.0]
      From worker 2:	[120.0, 118.0]
      From worker 3:	[128.0, 96.0]
  

Excessive output truncated after 524292 bytes.

      From worker 2:	[129.0, 126.0]
      From worker 2:	[129.0, 127.0]
      From worker 2:	[130.0, 127.0]
      From worker 4:	[139.0, 264.0]
      From worker 2:	[131.0, 127.0]
      From worker 3:	[129.0, 101.0]
      From worker 3:	[128.0, 101.0]
      From worker 3:	[128.0, 100.0]
      From worker 2:	[132.0, 127.0]
      From worker 4:	[139.0, 263.0]
      From worker 4:	[139.0, 262.0]
      From worker 4:	[139.0, 261.0]
      From worker 4:	[139.0, 262.0]
      From worker 3:	[129.0, 100.0]
      From worker 3:	[130.0, 100.0]
      From worker 4:	[139.0, 263.0]
      From worker 2:	[132.0, 126.0]
      From worker 3:	[131.0, 100.0]
      From worker 4:	[138.0, 263.0]
      From worker 3:	[131.0, 101.0]
      From worker 2:	[133.0, 126.0]
      From worker 4:	[138.0, 264.0]
      From worker 2:	[132.0, 126.0]
      From worker 4:	[137.0, 264.0]
      From worker 3:	[131.0, 102.0]
      From worker 4:	[136.0, 264.0]
      From worker 2:	[133.0, 126.0]
      From worker 3:	[130.0,

In [ ]:
sims_qntl = quantiles(simulations, [0.025,0.25,0.5,0.75,0.975]) ;

In [ ]:
mkpath("Simulations")

In [ ]:
writedlm("Simulations/CN_qnt_gill_jl.txt", sims_qntl[:,:,1])
writedlm("Simulations/ML_qnt_gill_jl.txt", sims_qntl[:,:,2])

In [ ]:
function distributions_t(sims, t, Tsim, δt)
    t_tot = [δt:δt:Tsim;]
    Nsim = size(sims)[3]
    nt = length(t)
    sim_t = Array{Float64}(undef, Nsim,nt,2)
    for i=1:Nsim
        for j=1:nt
            sim_t[i,j,1] = sims[findall(x->x==t[j], t_tot),1,i][1]
            sim_t[i,j,2] = sims[findall(x->x==t[j], t_tot),2,i][1]
        end
    end
    sim_t
end


In [ ]:
dist_sims = distributions_t(simulations, [10:10:80;]*365*24*3600, Tmax, step_out) ; 

In [ ]:
writedlm("Simulations/CN_ts_gill_jl.txt", dist_sims[:,:,1])
writedlm("Simulations/ML_ts_gill_jl.txt", dist_sims[:,:,2])

In [ ]:
"""
Time taken 
"""

In [ ]:
@everywhere function par_times(Nsim, f, SPN, Tmax, dtout)
    np = nworkers()            # Number of processes available.
    Nout = trunc(Int, Tmax/dtout) + 1 # dimension for output
    output = Array{Float64}(undef, Nsim) # Where we will write the results. As we do not know
                             # the type (Integer, Tuple...) we write "Any"
    i = 1
    nextidx() = (idx = i; i += 1; idx) # Function to know which is the next work item.
                                       # In this case it is just an index.
    @sync begin #@sync: must complete all jobs in block
        for p = 1:np # loops through all processes (workers)
            if p != myid() || np == 1 # first worker used only if all others are busy 
                @async begin # launch several tasks simultaneaously
                    while true
                        idx = nextidx()
                        if idx > Nsim
                            break
                        end
                        output[idx] = @elapsed remotecall_fetch(f, p, SPN, Tmax, dtout)
                    end
                end
            end
        end
    end
    output
end

In [ ]:
sim_times = par_times(Nsim, gillespied, Ntest, Tmax, step_out) 

mkpath("Simulations")
writedlm("Simulations/gill_times.txt", sim_times)

In [ ]:
sim_times

In [ ]:
dens = kde(sim_times)
density_points = hcat(dens.x, dens.density) ;

In [ ]:
plot(density_points[:,1], density_points[:,2], legend=false, title="Time Taken For One Gillespie Simulation" )